# Trainining and Validation Datasets & losses

Prepare de dataset for training 
Calculate losses

In [2]:
import torch
import myllm.gpt as gpt
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
# load vocabulary for learning purposes

file_path = "the-verdict.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()

total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("chars", total_characters)
print("tokens", total_tokens)

chars 20479
tokens 5145
